In [1]:
/ Load the table
d:("DTSFFFFF";enlist ",") 0:`$"daily.csv"
h:("DTSFFFFF";enlist ",") 0:`$"hourly.csv"
d: `sym`date xasc d
h: `sym`date`time xasc h
\c 100 1000

In [2]:
/Defines functions to calculate moving averages (EMA) and MACD indicators.
MA:{[x;n] n mavg x};
EMA:{[x;n] ema[2%(n+1);x]};
MACD:{[x;nFast;nSlow;nSig] diff:EMA[x;nFast]-EMA[x;nSlow]; sig:EMA[diff;nSig]; diff - sig};

cross_signal:{[m]
    m: update signalside:?[signal>0;1i;-1i], j:sums 1^i - prev i by sym from m;
    m: update signalidx:fills ?[0= deltas signalside;0N;j] by sym from m;
    update n:sums abs signalside, signaltime:first time, signalprice:first close by sym,signalidx from m
    }; 
 
cross_signal_bench:{[m]
    r: select from cross_signal[m] where n=1, 1 = abs signalside ;
    r: r upsert 0!select by sym from m; 
    r:update bps:10000*signalside*-1+pxexit%pxenter, nholds:(next j)-j by sym from update pxexit:next pxenter by sym from `sym`time xasc r;
    delete from r where null signalside
    };

In [3]:
/Defines functions to generate trading signals and calculate performance metrics for EMA and MACD crossover strategies on daily and hourly data.
ema_cross_over:{[data; ival; jval]
    data: update emaS:EMA[close;ival], emaL:EMA[close;jval] by sym from data;
    result: cross_signal_bench[update time:date, signal:emaS-emaL, pxenter:next open by sym from data];
    result: update ival:ival,jval:jval from result;
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by ival, jval, sym from result;
    res
    }

macd_cross_over:{[data; ival; jval; kval]
    data: update macd:MACD[close;ival;jval;kval] by sym from data;
    result:cross_signal_bench[update time:date, signal:macd, pxenter:next open by sym from data];
    result:update ival:ival, jval:jval, kval:kval from result;
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by ival,jval,kval,sym from result;
    res
    }

In [4]:
ema_cross_over_hourly:{[data; ival; jval]
    data: update emaS:EMA[close;ival], emaL:EMA[close;jval] by sym from data;
    result: cross_signal_bench[update signal:emaS-emaL, pxenter:next open by sym from data];
    result: update ival:ival,jval:jval from result;
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by ival, jval,sym from result;
    res
    }

macd_cross_over_hourly:{[data; ival; jval; kval]
    data: update macd:MACD[close;ival;jval;kval] by sym from data;
    result:cross_signal_bench[update signal:macd, pxenter:next open by sym from data];
    result:update ival:ival, jval:jval, kval:kval from result;
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by ival,jval,kval,sym from result;
    res
    }

In [6]:
/Sets up parameter ranges for EMA and MACD strategies
/Iterates over the parameter sets and applies the EMA and MACD crossover strategies to daily and hourly data, storing the results.
firstPos: 1 + 3 * til 30
secondPos: 3 + 3 * til 60

ema_params: raze {x,'/:y where x </: y}[;secondPos] each firstPos

temp: d
ema_daily_res:();
counter:0;
while[counter < count ema_params;
    cur: ema_params[counter];
    ival: cur 0;
    jval: cur 1;
    temp_res: ema_cross_over[temp;ival;jval];
    ema_daily_res:: ema_daily_res uj temp_res;
    counter+: 1;];

temp2: h
ema_hourly_res:();
counter:0;
while[counter < count ema_params;
    cur: ema_params[counter];
    ival: cur 0;
    jval: cur 1;
    temp_res: ema_cross_over_hourly[temp2;ival;jval];
    ema_hourly_res:: ema_hourly_res uj temp_res;
    counter+: 1;];

save `ema_daily_res.csv
save `ema_hourly_res.csv

`:ema_daily_res.csv


`:ema_hourly_res.csv


In [7]:
firstPos: 5 + til 10;
secondPos: 20 + til 11;
thirdPos: 5 + til 7;

macd_params: firstPos cross secondPos cross thirdPos;

temp: d
macd_daily_res:();
counter:0;
while[counter < count macd_params;
    cur: macd_params[counter];
    ival: cur 0;
    jval: cur 1;
    kval: cur 2;
    temp_res: macd_cross_over[temp;ival;jval;kval];
    macd_daily_res:: macd_daily_res uj temp_res;
    counter+: 1;];

temp2: h
macd_hourly_res:();
counter:0;
while[counter < count macd_params;
    cur: macd_params[counter];
    ival: cur 0;
    jval: cur 1;
    kval: cur 2;
    temp_res: macd_cross_over_hourly[temp2;ival;jval;kval];
    macd_hourly_res:: macd_hourly_res uj temp_res;
    counter+: 1;];

save `macd_daily_res.csv
save `macd_hourly_res.csv

`:macd_daily_res.csv


`:macd_hourly_res.csv


In [8]:
/ Updates the ema_daily_res and ema_hourly_res tables with the calculated high-risk (HR) and low-risk (LR) scores for EMA strategy.
ema_daily_res: update score_hr:0.3 * bps%10000 + 0.2 * rtn_sum + 0.1 * winpct + 0.3 * winmax + 0.1 * maxloss,
                        score_lr:0.1 * bps%10000 + 0.1 * rtn_sum + 0.4 * winpct + 0.1 * winmax + 0.3 * maxloss
               from ema_daily_res
ema_hourly_res: update score_hr:0.3 * bps%10000 + 0.2 * rtn_sum + 0.1 * winpct + 0.3 * winmax + 0.1 * maxloss,
                        score_lr:0.1 * bps%10000 + 0.1 * rtn_sum + 0.4 * winpct + 0.1 * winmax + 0.3 * maxloss
               from ema_hourly_res
save `ema_daily_res.csv
save `ema_hourly_res.csv

/Selects the optimal EMA parameter sets for daily and hourly data based on the maximum HR and LR scores.
hr_ema_daily: select ival, jval, sym, score_hr from ema_daily_res where score_hr=(max;score_hr) fby sym
lr_ema_daily: select ival, jval, sym, score_lr from ema_daily_res where score_lr=(max;score_lr) fby sym
hr_ema_hourly: select ival, jval, sym, score_hr from ema_hourly_res where score_hr=(max;score_hr) fby sym
lr_ema_hourly: select ival, jval, sym, score_lr from ema_hourly_res where score_lr=(max;score_lr) fby sym

hr_ema_daily
lr_ema_daily
hr_ema_hourly
lr_ema_hourly

`:ema_daily_res.csv


`:ema_hourly_res.csv


ival jval sym      score_hr 
----------------------------
22   69   BTC/USDT 0.1085098
88   180  ETH/USDT 0.3113048


ival jval sym      score_lr  
-----------------------------
22   69   BTC/USDT 0.03617241
88   180  ETH/USDT 0.1037814 


ival jval sym      score_hr  
-----------------------------
73   108  BTC/USDT 0.03126891
88   177  ETH/USDT 0.03550245


ival jval sym      score_lr  
-----------------------------
73   108  BTC/USDT 0.01042454
88   177  ETH/USDT 0.01183545


In [9]:
/ Updates the macd_daily_res and macd_hourly_res tables with the calculated high-risk (HR) and low-risk (LR) scores for MACD strategy.
macd_daily_res: update score_hr:0.3 * bps%10000 + 0.2 * rtn_sum + 0.1 * winpct + 0.3 * winmax + 0.1 * maxloss,
                        score_lr:0.1 * bps%10000 + 0.1 * rtn_sum + 0.4 * winpct + 0.1 * winmax + 0.3 * maxloss
               from macd_daily_res
macd_hourly_res: update score_hr:0.3 * bps%10000 + 0.2 * rtn_sum + 0.1 * winpct + 0.3 * winmax + 0.1 * maxloss,
                        score_lr:0.1 * bps%10000 + 0.1 * rtn_sum + 0.4 * winpct + 0.1 * winmax + 0.3 * maxloss
               from macd_hourly_res
save `macd_daily_res.csv
save `macd_hourly_res.csv

/Selects the optimal MACD parameter sets for daily and hourly data based on the maximum HR and LR scores.
hr_macd_daily: select ival, jval, kval, sym, score_hr from macd_daily_res where score_hr=(max;score_hr) fby sym
lr_macd_daily: select ival, jval, kval, sym, score_lr from macd_daily_res where score_lr=(max;score_lr) fby sym
hr_macd_daily
lr_macd_daily

hr_macd_hourly: select ival, jval, kval, sym, score_hr from macd_hourly_res where score_hr=(max;score_hr) fby sym
lr_macd_hourly: select ival, jval, kval, sym, score_lr from macd_hourly_res where score_lr=(max;score_lr) fby sym
hr_macd_hourly
lr_macd_hourly


`:macd_daily_res.csv


`:macd_hourly_res.csv


ival jval kval sym      score_hr  
----------------------------------
11   29   11   ETH/USDT 0.01159892
14   26   11   BTC/USDT 0.01156493


ival jval kval sym      score_lr   
-----------------------------------
11   29   11   ETH/USDT 0.00386651 
14   26   11   BTC/USDT 0.003855156


ival jval kval sym      score_hr   
-----------------------------------
12   24   9    BTC/USDT 0.002525256
14   29   10   ETH/USDT 0.001951637


ival jval kval sym      score_lr    
------------------------------------
12   24   9    BTC/USDT 0.0008418582
14   29   10   ETH/USDT 0.0006505996


In [10]:
/ Subsitute EMA optimal parameter sets for daily and hourly data
data: d
ema_cross_over[data; 22; 69]
ema_cross_over[data; 88; 180]

data: h
ema_cross_over_hourly[data; 73;108]
ema_cross_over_hourly[data; 88;177]

ival jval sym     | n  bps      rtn_sum  rtn_prd   duration winpct    winmax   maxloss   
------------------| ---------------------------------------------------------------------
22   69   BTC/USDT| 20 3617.503 6.873256 15.17416  104      0.55      4.385943 -0.4554534
22   69   ETH/USDT| 26 4329.71  10.82427 0.6652527 79       0.3076923 10.99666 -0.610821 


ival jval sym     | n  bps       rtn_sum  rtn_prd     duration winpct    winmax   maxloss   
------------------| ------------------------------------------------------------------------
88   180  BTC/USDT| 16 -216.8667 -0.3253  -0.9890614  135.6667 0.3125    2.399135 -0.9356136
88   180  ETH/USDT| 13 10379.51  12.45541 -0.06543847 136.4167 0.2307692 14.29928 -0.5316428


ival jval sym     | n   bps       rtn_sum  rtn_prd   duration winpct    winmax   maxloss  
------------------| ----------------------------------------------------------------------
73   108  BTC/USDT| 146 1042.615  15.11791 -2.12049  126.5241 0.5547945 2.62585  -1.983777
73   108  ETH/USDT| 134 -262.6692 -3.4935  -1.000004 135.4135 0.4925373 3.190109 -2.53862 


ival jval sym     | n   bps      rtn_sum   rtn_prd    duration winpct    winmax   maxloss  
------------------| -----------------------------------------------------------------------
88   177  BTC/USDT| 104 71.14879 0.7328326 -1         32.84466 0.4615385 1.984156 -1.525214
88   177  ETH/USDT| 94  1183.68  11.00822  -0.9977795 181.0108 0.5106383 4.879121 -2.650714


In [11]:
/ Subsitute MACD optimal parameter sets for daily and hourly data
data: d
macd_cross_over[data; 11; 29; 11] /eth
macd_cross_over[data; 14; 26; 11] /btc

data: h
macd_cross_over_hourly[data; 12; 24; 9] /btc
macd_cross_over_hourly[data; 14; 29; 10] /eth

ival jval kval sym     | n   bps      rtn_sum  rtn_prd  duration winpct    winmax   maxloss   
-----------------------| ---------------------------------------------------------------------
11   29   11   BTC/USDT| 136 317.2111 4.28235  10.54786 15.07407 0.3970588 1.020053 -0.2440921
11   29   11   ETH/USDT| 138 386.6722 5.297409 8.06098  14.85401 0.384058  1.101045 -0.5042322


ival jval kval sym     | n   bps      rtn_sum  rtn_prd  duration winpct    winmax   maxloss   
-----------------------| ---------------------------------------------------------------------
14   26   11   BTC/USDT| 124 385.5347 4.742076 15.98394 16.54472 0.3951613 1.020053 -0.2577632
14   26   11   ETH/USDT| 134 336.0538 4.469515 3.262553 15.30075 0.380597  1.071191 -0.5042322


ival jval kval sym     | n    bps      rtn_sum   rtn_prd    duration winpct    winmax    maxloss   
-----------------------| --------------------------------------------------------------------------
12   24   9    BTC/USDT| 1508 84.19671 12.68844  -0.9948473 12.31719 0.5218833 1.217145  -0.7771998
12   24   9    ETH/USDT| 1548 3.065932 0.4742997 -1         12.35553 0.4909561 0.6249045 -1.353218 


ival jval kval sym     | n    bps      rtn_sum   rtn_prd duration winpct    winmax   maxloss   
-----------------------| ----------------------------------------------------------------------
14   29   10   BTC/USDT| 1314 4.53536  0.5954928 -1      14.39299 0.5053272 1.041285 -0.7557188
14   29   10   ETH/USDT| 1290 65.06556 8.386951  -1      14.79131 0.5255814 0.815026 -1.096046 


In [12]:
/benchmark
select -1+(last close)% first close by sym from d
select -1+(last close)% first close by sym from h

sym     | x       
--------| --------
BTC/USDT| 4.833912
ETH/USDT| 4.542152


sym     | x         
--------| ----------
BTC/USDT| -0.1416338
ETH/USDT| 1.265533  


In [13]:
/ define a function to calculate the performance metrics of EMA strategies on a yearly basis.
ema_res_by_year:{[data; ival; jval]
    data: update emaS:EMA[close;ival], emaL:EMA[close;jval] by sym from data;
    result: cross_signal_bench[update time:date, signal:emaS-emaL, pxenter:next open by sym from data];
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by sym, date.year from result;
    res
    }
data: d
ema_res_by_year[data; 22; 69] /btc
ema_res_by_year[data; 88; 180] /eth
data: h
ema_res_by_year[data; 73; 108] /btc
ema_res_by_year[data; 88; 177] /eth


sym      year| n bps       rtn_sum    rtn_prd    duration winpct    winmax    maxloss    
-------------| --------------------------------------------------------------------------
BTC/USDT 2017| 4 956.2709  0.3825084  -0.2161549 41.5     0.25      1.204741  -0.4554534 
BTC/USDT 2018| 5 225.9281  0.112964   0.02155447 82       0.6       0.3624371 -0.2066578 
BTC/USDT 2019| 2 7564.28   1.512856   1.642147   152.5    1         1.421929  0.0909268  
BTC/USDT 2020| 3 13963.76  4.189128   3.377839   162.3333 0.3333333 4.385943  -0.1049251 
BTC/USDT 2021| 3 1088.366  0.3265098  0.3594819  105      1         0.1775349 0.07164975 
BTC/USDT 2022| 2 1746.45   0.34929    0.2845167  146.5    0.5       0.48331   -0.1340201 
BTC/USDT 2023| 1           0          0                   0         -0w       0w         
ETH/USDT 2017| 6 34.48942  0.02069365 -0.5301304 33.66667 0.1666667 1.231883  -0.5574194 
ETH/USDT 2018| 3 1027.263  0.3081788  0.06626164 117.6667 0.3333333 0.6915934 -0.343389  
ETH/USDT 2

sym      year| n bps       rtn_sum    rtn_prd    duration winpct    winmax     maxloss   
-------------| --------------------------------------------------------------------------
BTC/USDT 2017| 4 -1682.573 -0.6730291 -0.7382906 59.75    0.25      0.4480688  -0.7063905
BTC/USDT 2018| 3 -365.8953 -0.1097686 -0.1697218 130.6667 0.3333333 0.2641247  -0.2521758
BTC/USDT 2019| 2 -977.3006 -0.1954601 -0.2444362 136      0.5       0.144194   -0.3396541
BTC/USDT 2020| 3 3260.468  0.9781405  -0.8873715 174      0.3333333 2.399135   -0.9356136
BTC/USDT 2021| 2 -3241.334 -0.6482667 -0.5528941 96.5     0         -0.2256964 -0.4225704
BTC/USDT 2022| 1 3230.841  0.3230841  0.3230841  417      1         0.3230841  0.3230841 
BTC/USDT 2023| 1           0          0                   0         -0w        0w        
ETH/USDT 2017| 6 -634.6387 -0.3807832 -0.4268898 50       0.1666667 0.4365521  -0.3081395
ETH/USDT 2018| 1 4298.694  0.4298694  0.4298694  349      1         0.4298694  0.4298694 
ETH/USDT 2

sym      year| n  bps       rtn_sum     rtn_prd    duration winpct     winmax    maxloss    
-------------| -----------------------------------------------------------------------------
BTC/USDT 2021| 75 -10.23046 -0.07672848 -0.2784274 120.44   0.24       0.3917736 -0.2048336 
BTC/USDT 2022| 66 -70.83834 -0.4675331  -0.4553097 129.0455 0.1969697  0.308368  -0.1066656 
BTC/USDT 2023| 5  1231.176  0.4924703   0.5445032  418.25   0.8        0.3648298 0.001508498
ETH/USDT 2021| 61 38.70338  0.2360906   -0.2409674 144.9672 0.3278689  0.5352767 -0.1929655 
ETH/USDT 2022| 62 64.31356  0.3987441   0.1635483  140.3226 0.3064516  0.3450097 -0.1550517 
ETH/USDT 2023| 11 -92.52968 -0.09252968 -0.1242478 167.4    0.09090909 0.2704213 -0.08218828


sym      year| n  bps       rtn_sum    rtn_prd    duration winpct    winmax    maxloss    
-------------| ---------------------------------------------------------------------------
BTC/USDT 2021| 53 8.140481  0.04314455 -0.215331  178.566  0.245283  0.3848552 -0.1743967 
BTC/USDT 2022| 46 -51.73199 -0.2379672 -0.2946208 176.5217 0.2391304 0.284818  -0.1015482 
BTC/USDT 2023| 5  546.6988  0.2186795  0.1779998  411.75   0.2       0.3562174 -0.05632491
ETH/USDT 2021| 41 346.6377  1.421215   1.537783   222.561  0.3658537 0.4890207 -0.1600295 
ETH/USDT 2022| 44 120.4321  0.5299013  0.3519374  191.5682 0.3409091 0.3288623 -0.1348283 
ETH/USDT 2023| 9  -166.5928 -0.1332742 -0.1686041 209.25   0.1111111 0.2946482 -0.09975357


In [14]:
/ define a function to calculate the performance metrics of MACD strategies on a yearly basis.
macd_res_year:{[data; ival; jval; kval]
    data: update macd:MACD[close;ival;jval;kval] by sym from data;
    result:cross_signal_bench[update time:date, signal:macd, pxenter:next open by sym from data];
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by sym, date.year from result;
    res
    }
data: d
macd_res_year[data;14; 26; 11] /btc
macd_res_year[data;11; 29; 11] /eth
data:h
macd_res_year[data;12; 24; 9] /btc
macd_res_year[data;14; 29; 10] /eth


sym      year| n  bps       rtn_sum    rtn_prd    duration winpct    winmax    maxloss    
-------------| ---------------------------------------------------------------------------
BTC/USDT 2017| 9  1338.614  1.204752   1.163775   15.88889 0.5555556 1.020053  -0.1809949 
BTC/USDT 2018| 23 595.1905  1.368938   1.9728     16.13043 0.5217391 0.4196563 -0.1060576 
BTC/USDT 2019| 26 146.2072  0.3801386  0.1526004  14.46154 0.3461538 0.4674527 -0.1677552 
BTC/USDT 2020| 16 1116.049  1.785679   2.685641   22.3125  0.4375    0.7218905 -0.2440921 
BTC/USDT 2021| 26 -40.72698 -0.1058901 -0.3165204 13.65385 0.2307692 0.3844138 -0.1429166 
BTC/USDT 2022| 19 -67.68229 -0.1285964 -0.2296254 19.10526 0.4736842 0.2982719 -0.2577632 
BTC/USDT 2023| 5  592.6368  0.2370547  0.1804332  17.5     0.2       0.4086006 -0.06411178
ETH/USDT 2017| 9  -757.0366 -0.6813329 -0.6508931 15.44444 0.4444444 0.2982732 -0.5042322 
ETH/USDT 2018| 21 653.555   1.372465   1.137423   17.7619  0.3809524 0.7741902 -0.2246523 

sym      year| n  bps       rtn_sum    rtn_prd     duration winpct    winmax    maxloss    
-------------| ----------------------------------------------------------------------------
BTC/USDT 2017| 11 953.834   1.049217   0.8346134   12.90909 0.5454545 1.020053  -0.1809949 
BTC/USDT 2018| 25 405.9946  1.014987   1.115673    14.84    0.56      0.4025851 -0.1269641 
BTC/USDT 2019| 28 99.18621  0.2777214  0.07773239  13.46429 0.3571429 0.3761979 -0.1677552 
BTC/USDT 2020| 18 838.5984  1.509477   1.803888    19.83333 0.4444444 0.7218905 -0.2440921 
BTC/USDT 2021| 26 73.35438  0.1907214  -0.0723043  13.65385 0.2307692 0.3844138 -0.1429166 
BTC/USDT 2022| 23 18.05533  0.04152725 -0.07064461 15.78261 0.3913043 0.3179349 -0.1536149 
BTC/USDT 2023| 5  496.748   0.1986992  0.1419563   17.5     0.2       0.3867603 -0.06516374
ETH/USDT 2017| 9  -631.8729 -0.5686856 -0.6149762  15.44444 0.5555556 0.3619895 -0.5042322 
ETH/USDT 2018| 21 739.659   1.553284   1.408461    17.71429 0.3809524 0.8935947 

sym      year| n   bps       rtn_sum    rtn_prd    duration winpct    winmax     maxloss    
-------------| -----------------------------------------------------------------------------
BTC/USDT 2021| 672 11.7267   0.7880345  0.5761896  13.00595 0.3452381 0.1960017  -0.09832867
BTC/USDT 2022| 696 -6.619934 -0.4607474 -0.4770086 12.59052 0.3275862 0.1341587  -0.08684862
BTC/USDT 2023| 140 5.966479  0.08293406 0.0530874  12.84892 0.3571429 0.09512312 -0.06934332
ETH/USDT 2021| 724 5.896151  0.4268813  -0.1026785 12.07182 0.3577348 0.2498475  -0.1603112 
ETH/USDT 2022| 692 5.765309  0.3989594  0.05920609 12.66618 0.3482659 0.1740162  -0.1293941 
ETH/USDT 2023| 132 7.454662  0.09765608 0.06842903 13.58015 0.3409091 0.07881378 -0.05980583


sym      year| n   bps       rtn_sum     rtn_prd     duration winpct    winmax     maxloss    
-------------| -------------------------------------------------------------------------------
BTC/USDT 2021| 590 16.38349  0.966626    0.9377983   14.81356 0.3728814 0.1828914  -0.1151185 
BTC/USDT 2022| 608 -1.123853 -0.06833026 -0.2191652  14.41447 0.3355263 0.1372285  -0.08684862
BTC/USDT 2023| 116 3.058427  0.03517191  0.003853684 15.53043 0.362069  0.09681398 -0.06233469
ETH/USDT 2021| 602 18.09483  1.089309    0.758879    14.51827 0.3654485 0.3250727  -0.1489989 
ETH/USDT 2022| 564 16.29768  0.9191891   0.7827115   15.54078 0.3652482 0.1992899  -0.1293941 
ETH/USDT 2023| 124 -7.896735 -0.09712984 -0.1190481  14.50407 0.3145161 0.07479175 -0.05345256


In [18]:
/ define a function to calculate the performance metrics of EMA strategies on a yearly basis.
ema_res_by_side:{[data; ival; jval]
    data: update emaS:EMA[close;ival], emaL:EMA[close;jval] by sym from data;
    result: cross_signal_bench[update time:date, signal:emaS-emaL, pxenter:next open by sym from data];
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by sym,signalside from result;
    res
    }
data: d
ema_res_by_side[data; 22; 69] /btc
ema_res_by_side[data; 88; 180] /eth
data: h
ema_res_by_side[data; 73; 108] /btc
ema_res_by_side[data; 88; 177] /eth

sym      signalside| n  bps      rtn_sum    rtn_prd    duration winpct    winmax    maxloss   
-------------------| -------------------------------------------------------------------------
BTC/USDT -1        | 10 572.7529 0.5727529  0.2967847  105.3    0.7       0.48331   -0.4554534
BTC/USDT 1         | 10 7000.56  6.300504   11.47251   102.5556 0.4       4.385943  -0.3015673
ETH/USDT -1        | 13 -409.462 -0.5323006 -0.7699328 75       0.3076923 0.6915934 -0.610821 
ETH/USDT 1         | 13 9463.813 11.35658   6.238114   83.33333 0.3076923 10.99666  -0.343389 


sym      signalside| n bps       rtn_sum   rtn_prd    duration winpct    winmax    maxloss   
-------------------| ------------------------------------------------------------------------
BTC/USDT -1        | 8 -2704.582 -2.163665 -0.9918358 125.5    0.25      0.3230841 -0.9356136
BTC/USDT 1         | 8 2626.236  1.838365  0.3398183  147.2857 0.375     2.399135  -0.4853807
ETH/USDT -1        | 7 -1867.902 -1.120741 -0.8190676 100.6667 0.1428571 0.4298694 -0.5316428
ETH/USDT 1         | 6 22626.91  13.57615  4.165252   172.1667 0.3333333 14.29928  -0.4774829


sym      signalside| n  bps      rtn_sum    rtn_prd    duration winpct    winmax    maxloss   
-------------------| -------------------------------------------------------------------------
BTC/USDT -1        | 73 -18.4307 -0.1345441 -0.2901524 137.3425 0.2739726 0.308368  -0.2048336
BTC/USDT 1         | 73 11.49346 0.08275289 -0.1448282 127.7361 0.2054795 0.3917736 -0.1057855
ETH/USDT -1        | 67 -80.5262 -0.5395255 -0.5666367 135.4776 0.3283582 0.3450097 -0.1929655
ETH/USDT 1         | 67 163.9137 1.081831   0.7847358  153.6364 0.2686567 0.5352767 -0.1591493


sym      signalside| n  bps       rtn_sum    rtn_prd     duration winpct    winmax    maxloss    
-------------------| ----------------------------------------------------------------------------
BTC/USDT -1        | 52 -28.13927 -0.1463242 -0.2938941  194.3654 0.2307692 0.3083305 -0.1743967 
BTC/USDT 1         | 52 33.36885  0.1701811  -0.07661142 178.902  0.25      0.3848552 -0.09210318
ETH/USDT -1        | 47 18.49287  0.08691647 -0.1486731  195.5745 0.3404255 0.3288623 -0.1600295 
ETH/USDT 1         | 47 376.2881  1.730925   2.3506      218.1739 0.3191489 0.4890207 -0.1139241 


In [21]:
/ define a function to calculate the performance metrics of EMA strategies by side.
macd_res_side:{[data; ival; jval; kval]
    data: update macd:MACD[close;ival;jval;kval] by sym from data;
    result:cross_signal_bench[update time:date, signal:macd, pxenter:next open by sym from data];
    res: select n:count i, avg bps, rtn_sum:sum bps%10000, rtn_prd:-1+prd 1+bps%10000, duration:avg nholds, winpct:(count i where bps>0)%count i,winmax:max bps%10000, maxloss:min bps%10000  by sym,signalside  from result;
    res
    }
data: d
macd_res_side[data;14; 26; 11] /btc
macd_res_side[data;11; 29; 11] /eth
data:h
macd_res_side[data;12; 24; 9] /btc
macd_res_side[data;14; 29; 10] /eth

sym      signalside| n  bps       rtn_sum    rtn_prd     duration winpct    winmax    maxloss   
-------------------| ---------------------------------------------------------------------------
BTC/USDT -1        | 62 74.28592  0.4605727  -0.06758649 15.09677 0.3548387 0.4196563 -0.2440921
BTC/USDT 1         | 62 701.8858  4.281504   17.21503    18.01639 0.4354839 1.020053  -0.2577632
ETH/USDT -1        | 67 -29.29709 -0.1962905 -0.7457758  14.44776 0.3432836 0.520461  -0.5042322
ETH/USDT 1         | 67 706.9403  4.665806   15.7669     16.16667 0.4179104 1.071191  -0.2246523


sym      signalside| n  bps      rtn_sum   rtn_prd    duration winpct    winmax    maxloss   
-------------------| ------------------------------------------------------------------------
BTC/USDT -1        | 68 40.71746 0.2768788 -0.2287825 13.86765 0.3823529 0.4025851 -0.2440921
BTC/USDT 1         | 68 597.8316 4.005472  13.97354   16.29851 0.4117647 1.020053  -0.1536149
ETH/USDT -1        | 69 25.96208 0.1791383 -0.6286482 13.85507 0.3623188 0.520461  -0.5042322
ETH/USDT 1         | 69 752.6869 5.118271  23.39999   15.86765 0.4057971 1.101045  -0.2343355


sym      signalside| n   bps        rtn_sum     rtn_prd    duration winpct    winmax    maxloss    
-------------------| ------------------------------------------------------------------------------
BTC/USDT -1        | 754 0.05533354 0.004172149 -0.2273483 12.98011 0.3395225 0.1735983 -0.09832867
BTC/USDT 1         | 754 5.392417   0.406049    0.1235273  12.61886 0.33687   0.1960017 -0.08684862
ETH/USDT -1        | 774 -5.270954  -0.4079718  -0.5638328 12.59044 0.3397933 0.1804444 -0.1489989 
ETH/USDT 1         | 774 17.22469   1.331469    1.328205   12.34023 0.3643411 0.2498475 -0.1603112 


sym      signalside| n   bps      rtn_sum    rtn_prd    duration winpct    winmax    maxloss    
-------------------| ---------------------------------------------------------------------------
BTC/USDT -1        | 657 4.313592 0.283403   0.0329315  14.8691  0.3637747 0.1508403 -0.1151185 
BTC/USDT 1         | 657 9.909522 0.6500647  0.4705055  14.51372 0.3455099 0.1828914 -0.08684862
ETH/USDT -1        | 645 1.321261 0.08522131 -0.2837319 15.08837 0.3534884 0.2392714 -0.1489989 
ETH/USDT 1         | 645 28.35632 1.826147   2.856502   14.84006 0.3674419 0.3250727 -0.1293941 
